In [2]:
import pandas as pd
import numpy as np
import datetime
from yahoo_fin.stock_info import get_data
import statsmodels.formula.api as smf
from pypfopt.efficient_frontier import EfficientFrontier

In [3]:
#Output the top 10 CC
def top10MarketCapCC():
    import json
    from requests import Request, Session
    from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
    
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest?convert=USD'
    parameters = {
        'symbol' : 'ADA,ADX,AE,ANT,ARDR,ARK,BAT,BCH,BCN,BNB,BNT,BTC,BTG,BTS,CVC,DASH,DCR,DGB,DGD,DNT,DOGE,EDG,EOS,ETC,ETH,FUN,GAS,GBYTE,GNO,HC,ICX,KIN,KMD,KNC,LINK,LRC,LSK,LTC,MAID,MCO,MIOTA,MLN,MONA,MTL,NANO,NEO,NXS,OMG,PIVX,PPT,QRL,QTUM,REP,RLC,SALT,SC,SNT,STEEM,STORJ,SYS,TRX,VERI,VET,WAVES,WTC,XEM,XLM,XMR,XRP,XVG,ZEC,ZEN,ZRX'
    }
    headers = {
        'Accepts': 'application/json',
    #Input your API key here
        'X-CMC_PRO_API_KEY': '4797f5ee-9a2d-42d1-979a-ae14643c966d',
    }
    session = Session()
    session.headers.update(headers)
    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        result = data
        d = pd.DataFrame(result)
        f = d.iloc[6:,1]
        MarketCap_rank = pd.DataFrame(columns = ['Symbol', 'MarketCap'])
        for i in range(73):
            Append = {'Symbol': f.iloc[i]['symbol'], 'MarketCap' : f.iloc[i]['quote']['USD']['market_cap']}
            MarketCap_rank = MarketCap_rank.append(Append, ignore_index=True)
        MarketCap_rank = MarketCap_rank.sort_values(by=['MarketCap'], ascending = False)
        MarketCap_rank.reset_index(drop=True, inplace=True)
        MarketCap_rank = MarketCap_rank.truncate(before=None, after=9, copy=True)
        return MarketCap_rank['Symbol'] + '-USD'
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [4]:
#Output market returns
def MarketReturns():
    CC_list = pd.read_csv("WorldOfCC.csv")['List']
    data = pd.DataFrame()
    end = datetime.date.today()
    start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)
    for i in CC_list:
        data[i] = get_data(i, start, end,True, '1d')['adjclose']
    data= data.fillna(value=.001)#Usd because faulty API returns NaN values
    marketData = np.log(data/ data.shift(1)).fillna(value=0.001)
    sum_row = marketData.sum(axis=1)
    return sum_row
    

In [5]:
I = MarketReturns()

In [6]:
J = pd.DataFrame(I, columns = ['MarketReturns'])

In [7]:
#Load the list of CC used in the research
CC_list = top10MarketCapCC()

end = datetime.date.today()
start = datetime.datetime.now() - datetime.timedelta(days = 7 * 8)

data = pd.DataFrame()
for i in CC_list:
    data[i] = get_data(i, start, end,True, '1d')['adjclose']
data= data.fillna(value=.001)
marketData = np.log(data/ data.shift(1)).fillna(value=0.001)

In [8]:
result = pd.concat([marketData.rename(columns=lambda s: s.replace('-USD','')), J], axis=1)

In [9]:
result.mean()
result.std()

BTC               3.273165
ETH               2.663489
XRP               1.065455
ADA               1.105397
LINK              1.882046
LTC               2.237661
BCH               2.459697
BNB               2.011199
XLM               1.062636
EOS               1.491839
MarketReturns    96.253176
dtype: float64

In [10]:
Name = list(result.iloc[:,0:10])
regression = pd.DataFrame(columns=['beta', 'rse'])
for i in range(10):
    CAPM_model = smf.ols(formula = Name[i]+' ~ MarketReturns', data = result)
    CAPM_fit = CAPM_model.fit()
    beta = CAPM_fit.params['MarketReturns']
    rse = CAPM_fit.bse['MarketReturns']
    dict = {'beta': beta, 'rse': rse} 
    df = pd.DataFrame(dict, index = [Name[i]])
    regression = regression.append(df)
    
print(regression)

          beta       rse
BTC   0.033994  0.000119
ETH   0.027666  0.000073
XRP   0.011033  0.000122
ADA   0.011472  0.000072
LINK  0.019546  0.000072
LTC   0.023243  0.000060
BCH   0.025549  0.000072
BNB   0.020890  0.000064
XLM   0.011003  0.000121
EOS   0.015495  0.000048


In [11]:
def calculateReturns(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        x[i] = result.mean()['MarketReturns'] * regression.loc[y]['beta']
    return x

In [12]:
ExpectedReturns = pd.DataFrame(index = ['returns'],columns = list(result.iloc[:,0:10]))
ExpectedReturns = ExpectedReturns.apply(lambda x: calculateReturns(x), axis=1) 
ExpectedReturns.T

,returns
BTC,0.010781
ETH,0.008774
XRP,0.003499
ADA,0.003638
LINK,0.006199
LTC,0.007371
BCH,0.008102
BNB,0.006625
XLM,0.003489
EOS,0.004914


In [13]:
def calculateCovariance(x):
    for i in range(10):
        y = str(x[i:i+1]).split(' ')[0]
        if(y == x.name):
            x[i] = (result.std()['MarketReturns'] * regression.loc[y]['beta']) ** 2 + regression.loc[y]['rse']
        else:
            x[i] = (result.std()['MarketReturns'] ** 2) * (regression.loc[y]['beta'] * regression.loc[x.name]['beta'])
    return x

In [14]:
CovarianceMatrix = pd.DataFrame(index = list(result.iloc[:,0:10]),columns = list(result.iloc[:,0:10]))
CovarianceMatrix = CovarianceMatrix.apply(lambda x: calculateCovariance(x), axis=1) 
CovarianceMatrix 

,BTC,ETH,XRP,ADA,LINK,LTC,BCH,BNB,XLM,EOS
BTC,10.706563,8.713451,3.474663,3.613010,6.155874,7.320442,8.046563,6.579113,3.465463,4.880113
ETH,8.713451,7.091524,2.827858,2.940452,5.009964,5.957749,6.548703,5.354418,2.820370,3.971685
XRP,3.474663,2.827858,1.127787,1.172564,1.997824,2.375772,2.611427,2.135181,1.124679,1.583789
ADA,3.613010,2.940452,1.172564,1.219323,2.077369,2.470366,2.715403,2.220196,1.169459,1.646849
LINK,6.155874,5.009964,1.997824,2.077369,3.539509,4.209027,4.626524,3.782786,1.992534,2.805914
LTC,7.320442,5.957749,2.375772,2.470366,4.209027,5.005352,5.501771,4.498414,2.369481,3.336737
BCH,8.046563,6.548703,2.611427,2.715403,4.626524,5.501771,6.047567,4.944615,2.604512,3.667711
BNB,6.579113,5.354418,2.135181,2.220196,3.782786,4.498414,4.944615,4.042930,2.129528,2.998831
XLM,3.465463,2.820370,1.124679,1.169459,1.992534,2.369481,2.604512,2.129528,1.121823,1.579596
EOS,4.880113,3.971685,1.583789,1.646849,2.805914,3.336737,3.667711,2.998831,1.579596,2.224457


In [15]:
ef1 = EfficientFrontier(ExpectedReturns.iloc[0], CovarianceMatrix, weight_bounds=(0, 1))
weights = ef1.max_sharpe(risk_free_rate= 0.0)
weights = ef1.clean_weights()

weights

OrderedDict([('BTC', 0.10733),
             ('ETH', 0.14168),
             ('XRP', 0.03397),
             ('ADA', 0.05952),
             ('LINK', 0.10121),
             ('LTC', 0.14489),
             ('BCH', 0.1335),
             ('BNB', 0.12309),
             ('XLM', 0.03396),
             ('EOS', 0.12085)])